# MNIST & Deeper CNN - 컨볼루션 신경망을 이용해 MNIST 데이터셋 분류하기

## Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

## Device & reproducibility


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

## Parameters

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# Data & Data Loader


In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download= True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download= True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size= batch_size,
                                          shuffle= True,
                                          drop_last= True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



# Model

In [5]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size= 3, stride =1, padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride= 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size= 3, stride= 1,padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride= 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size= 3, stride= 1, padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride= 2)
    )

    self.fc1 = nn.Linear(3*3*128, 625, bias=True)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(625, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)

    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [6]:
model = CNN().to(device)

value = torch.Tensor(1, 1, 28, 28).to(device)
print( (model(value)).shape )

torch.Size([1, 10])


## Cost function & optimizer

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

## Training

In [8]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    
    avg_cost += cost / total_batch
  
  print('Epoch: {}  Cost= {}'.format(epoch+1, avg_cost))
print('Learning Finished')

Epoch: 1  Cost= 0.16142261028289795
Epoch: 2  Cost= 0.04304741322994232
Epoch: 3  Cost= 0.030921127647161484
Epoch: 4  Cost= 0.0221807099878788
Epoch: 5  Cost= 0.018063558265566826
Epoch: 6  Cost= 0.0139582185074687
Epoch: 7  Cost= 0.012254936620593071
Epoch: 8  Cost= 0.012281147763133049
Epoch: 9  Cost= 0.009239986538887024
Epoch: 10  Cost= 0.008619440719485283
Epoch: 11  Cost= 0.007332132197916508
Epoch: 12  Cost= 0.007848759181797504
Epoch: 13  Cost= 0.005035451613366604
Epoch: 14  Cost= 0.00583705073222518
Epoch: 15  Cost= 0.005284077022224665
Learning Finished


## Testing

In [9]:
with torch.no_grad():
  X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)  
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.9896999597549438
